In [9]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil

#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('^EMAlong25', 'yahoo', start, end)
# data = web.DataReader('AAPL', 'yahoo', start, end)
data = data.astype("double")
print(data)

initial = 1000000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

             High    Low   Open  Close       Volume  Adj Close
Date                                                          
2021-09-14  363.0  356.0  361.0  358.0   66934700.0      358.0
2021-09-15  367.0  361.0  364.0  362.0   54868371.0      362.0
2021-09-16  371.0  360.0  360.0  367.0   52270565.0      367.0
2021-09-17  367.0  362.0  366.0  362.0   43735573.0      362.0
2021-09-21  379.0  373.0  378.0  378.0   83213264.0      378.0
...           ...    ...    ...    ...          ...        ...
2022-09-08  367.0  357.0  367.0  357.0  113510594.0      357.0
2022-09-09  357.0  352.0  354.0  353.0   99795902.0      353.0
2022-09-12  348.0  343.0  346.0  346.0   73706891.0      346.0
2022-09-13  346.0  342.0  345.0  344.0   46433034.0      344.0
2022-09-14  364.0  357.0  363.0  359.0   95315479.0      359.0

[246 rows x 6 columns]


**MACD CROSS**

In [10]:
import BacktestingStrategies.Strategy_MacdCross as btstMACD
importlib.reload(btstMACD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACD.MACDCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）

#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)
output2=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    maximize='Equity Final [$]',
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 


if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACD.MACDCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True, #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）

    #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)
    output2=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        maximize='Equity Final [$]',
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   42.276423
Equity Final [$]                 964173.51495
Equity Peak [$]                 1164152.95165
Return [%]                          -3.582649
Buy & Hold Return [%]                 0.27933
Return (Ann.) [%]                   -3.668408
Volatility (Ann.) [%]               28.140622
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.246128
Avg. Drawdown [%]                   -8.919992
Max. Drawdown Duration      189 days 00:00:00
Avg. Drawdown Duration       57 days 00:00:00
# Trades                                    7
Win Rate [%]                        42.857143
Best Trade [%]                       6.850226
Worst Trade [%]                     -6.401555
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2469,50,59,404.99485,397.0,-19739.28465,-0.019741,2021-11-29,2021-12-10,11 days
1,2482,79,97,394.94535,422.0,67149.64130,0.068502,2022-01-12,2022-02-07,26 days
2,2300,108,123,455.24235,436.0,-44257.40500,-0.042268,2022-02-24,2022-03-17,21 days
3,2422,139,163,414.03940,428.0,33812.57320,0.033718,2022-04-11,2022-05-19,38 days
4,2516,182,193,412.02950,403.0,-22718.22200,-0.021915,2022-06-15,2022-06-30,15 days
5,2397,195,200,423.08395,396.0,-64920.22815,-0.064016,2022-07-04,2022-07-11,7 days
6,2699,230,245,351.73250,363.0,30410.98250,0.032034,2022-08-24,2022-09-14,21 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   36.585366
Equity Final [$]                 971116.77265
Equity Peak [$]                 1175490.34635
Return [%]                          -2.888323
Buy & Hold Return [%]                 0.27933
Return (Ann.) [%]                   -2.957718
Volatility (Ann.) [%]               26.785679
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -21.709968
Avg. Drawdown [%]                   -9.434342
Max. Drawdown Duration      189 days 00:00:00
Avg. Drawdown Duration       60 days 00:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                       6.794599
Worst Trade [%]                     -5.325153
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2512,82,101,397.96020,425.0,67923.97760,0.067946,2022-01-17,2022-02-14,28 days
1,2542,103,104,420.06910,416.0,-10343.65220,-0.009687,2022-02-16,2022-02-17,1 days
2,2323,108,124,455.24235,431.0,-56314.97905,-0.053252,2022-02-24,2022-03-18,22 days
3,2424,150,166,413.03445,401.0,-29171.50680,-0.029137,2022-04-26,2022-05-24,28 days
4,2412,183,204,402.98495,399.0,-9611.69940,-0.009889,2022-06-16,2022-07-15,29 days
5,2675,234,245,359.77210,363.0,8634.63250,0.008972,2022-08-30,2022-09-14,15 days


<Strategy MACDCross(MACDshort=20,MACDlong=50,MACDsignal=10)>

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   79.674797
Equity Final [$]                1022328.43015
Equity Peak [$]                  1152582.2969
Return [%]                           2.232843
Buy & Hold Return [%]                 0.27933
Return (Ann.) [%]                    2.287921
Volatility (Ann.) [%]               37.856539
Sharpe Ratio                         0.060437
Sortino Ratio                        0.093552
Calmar Ratio                         0.091329
Max. Drawdown [%]                  -25.051458
Avg. Drawdown [%]                  -10.980278
Max. Drawdown Duration      132 days 00:00:00
Avg. Drawdown Duration       48 days 00:00:00
# Trades                                   14
Win Rate [%]                             50.0
Best Trade [%]                      11.176485
Worst Trade [%]                     -7.879977
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2469,50,59,404.99485,397.0,-19739.28465,-0.019741,2021-11-29,2021-12-10,11 days
1,-2481,59,79,395.03485,393.0,5048.46285,0.005151,2021-12-10,2022-01-12,33 days
2,2494,79,97,394.94535,422.0,67474.29710,0.068502,2022-01-12,2022-02-07,26 days
3,-2507,97,108,419.91110,453.0,-82953.87230,-0.078800,2022-02-07,2022-02-24,17 days
4,2130,108,123,455.24235,436.0,-40986.20550,-0.042268,2022-02-24,2022-03-17,21 days
5,-2140,123,139,433.84180,412.0,46741.45200,0.050345,2022-03-17,2022-04-11,25 days
6,2356,139,163,414.03940,428.0,32891.17360,0.033718,2022-04-11,2022-05-19,38 days
7,-2367,163,182,425.88140,410.0,37591.27380,0.037291,2022-05-19,2022-06-15,27 days
8,2538,182,193,412.02950,403.0,-22916.87100,-0.021915,2022-06-15,2022-06-30,15 days
9,-2551,193,195,401.00515,421.0,-51006.86235,-0.049862,2022-06-30,2022-07-04,4 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   71.544715
Equity Final [$]                 986728.74145
Equity Peak [$]                  1128627.0764
Return [%]                          -1.327126
Buy & Hold Return [%]                 0.27933
Return (Ann.) [%]                   -1.359274
Volatility (Ann.) [%]               35.574667
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -23.926883
Avg. Drawdown [%]                   -8.548887
Max. Drawdown Duration      230 days 00:00:00
Avg. Drawdown Duration       66 days 00:00:00
# Trades                                   12
Win Rate [%]                        41.666667
Best Trade [%]                       9.829344
Worst Trade [%]                     -9.435939
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2590,70,82,386.07940,396.0,-25694.35400,-0.025696,2021-12-27,2022-01-17,21 days
1,2448,82,101,397.96020,425.0,66193.43040,0.067946,2022-01-17,2022-02-14,28 days
2,-2460,101,103,422.89625,418.0,12044.77500,0.011578,2022-02-14,2022-02-16,2 days
3,2505,103,104,420.06910,416.0,-10193.09550,-0.009687,2022-02-16,2022-02-17,1 days
4,-2518,104,108,413.94080,453.0,-98351.06560,-0.094359,2022-02-17,2022-02-24,7 days
5,2073,108,124,455.24235,431.0,-50254.39155,-0.053252,2022-02-24,2022-03-18,22 days
6,-2083,124,150,428.86655,411.0,37216.02365,0.041660,2022-03-18,2022-04-26,39 days
7,2253,150,166,413.03445,401.0,-27113.61585,-0.029137,2022-04-26,2022-05-24,28 days
8,-2265,166,183,399.01505,401.0,-4495.91175,-0.004975,2022-05-24,2022-06-16,23 days
9,2231,183,204,402.98495,399.0,-8890.42345,-0.009889,2022-06-16,2022-07-15,29 days


<Strategy MACDCross_WithShortPosition(MACDshort=20,MACDlong=50,MACDsignal=10)>

**EMA CROSS**

In [26]:
import BacktestingStrategies.Strategy_EMACross as btstEMA
importlib.reload(btstEMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstEMA.EmaCrossStrategy, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") 

#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    EMAshort=range(10, 100, 10),\
    EMAlong=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.EMAlong > x.EMAshort),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstEMA.EmaCrossStrategy_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") 

    #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        EMAshort=range(10, 100, 10),\
        EMAlong=range(10, 200, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.EMAlong > x.EMAshort),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   41.869919
Equity Final [$]                   856155.935
Equity Peak [$]                   1145976.822
Return [%]                         -14.384407
Buy & Hold Return [%]                -0.27933
Return (Ann.) [%]                  -14.708094
Volatility (Ann.) [%]                24.42367
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -25.290292
Avg. Drawdown [%]                  -12.233572
Max. Drawdown Duration      189 days 00:00:00
Avg. Drawdown Duration       72 days 00:00:00
# Trades                                   10
Win Rate [%]                             20.0
Best Trade [%]                       6.850226
Worst Trade [%]                     -6.401555
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2469,50,59,404.99485,397.0,-19739.28465,-0.019741,2021-11-29,2021-12-10,11 days
1,2438,66,68,401.98000,389.0,-31645.24000,-0.032290,2021-12-21,2021-12-23,2 days
2,2401,79,97,394.94535,422.0,64958.21465,0.068502,2022-01-12,2022-02-07,26 days
3,2412,103,104,420.06910,416.0,-9814.66920,-0.009687,2022-02-16,2022-02-17,1 days
4,2264,106,124,443.18295,431.0,-27582.19880,-0.027490,2022-02-21,2022-03-18,25 days
5,2357,139,147,414.03940,399.0,-35447.86580,-0.036324,2022-04-11,2022-04-21,10 days
6,2277,150,163,413.03445,428.0,34076.55735,0.036233,2022-04-26,2022-05-19,23 days
7,2365,182,192,412.02950,402.0,-23719.76750,-0.024342,2022-06-15,2022-06-29,14 days
8,2247,195,200,423.08395,396.0,-60857.63565,-0.064016,2022-07-04,2022-07-11,7 days
9,2474,234,243,359.77210,346.0,-34072.17540,-0.038280,2022-08-30,2022-09-12,13 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    1.219512
Equity Final [$]                  990212.2944
Equity Peak [$]                  1012100.2944
Return [%]                          -0.978771
Buy & Hold Return [%]                -0.27933
Return (Ann.) [%]                   -1.002523
Volatility (Ann.) [%]                1.975842
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -2.162632
Avg. Drawdown [%]                   -2.162632
Max. Drawdown Duration      121 days 00:00:00
Avg. Drawdown Duration      121 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -0.979151
Worst Trade [%]                     -0.979151
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2432,160,162,411.02455,407.0,-9787.7056,-0.009792,2022-05-16,2022-05-18,2 days


<Strategy EmaCrossStrategy(EMAshort=40,EMAlong=60)>

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   91.056911
Equity Final [$]                 762386.25715
Equity Peak [$]                 1049269.30435
Return [%]                         -23.761374
Buy & Hold Return [%]                -0.27933
Return (Ann.) [%]                  -24.264189
Volatility (Ann.) [%]                30.18622
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -30.51127
Avg. Drawdown [%]                   -8.165146
Max. Drawdown Duration      302 days 00:00:00
Avg. Drawdown Duration       67 days 00:00:00
# Trades                                   21
Win Rate [%]                        28.571429
Best Trade [%]                       9.146233
Worst Trade [%]                     -6.536973
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2583,22,50,387.07445,403.0,-41135.69565,-0.041143,2021-10-18,2021-11-29,42 days
1,2367,50,59,404.99485,397.0,-18923.80995,-0.019741,2021-11-29,2021-12-10,11 days
2,-2379,59,66,395.03485,400.0,-11812.09185,-0.012569,2021-12-10,2021-12-21,11 days
3,2308,66,68,401.98000,389.0,-29957.84000,-0.032290,2021-12-21,2021-12-23,2 days
4,-2320,68,79,387.07445,393.0,-13747.27600,-0.015309,2021-12-23,2022-01-12,20 days
5,2239,79,97,394.94535,422.0,60575.36135,0.068502,2022-01-12,2022-02-07,26 days
6,-2250,97,103,419.91110,418.0,4299.97500,0.004551,2022-02-07,2022-02-16,9 days
7,2259,103,104,420.06910,416.0,-9192.09690,-0.009687,2022-02-16,2022-02-17,1 days
8,-2271,104,106,413.94080,441.0,-61451.44320,-0.065370,2022-02-17,2022-02-21,4 days
9,1982,106,124,443.18295,431.0,-24146.60690,-0.027490,2022-02-21,2022-03-18,25 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   44.308943
Equity Final [$]                1087394.64085
Equity Peak [$]                 1162786.64085
Return [%]                           8.739464
Buy & Hold Return [%]                -0.27933
Return (Ann.) [%]                    8.961903
Volatility (Ann.) [%]               27.153997
Sharpe Ratio                          0.33004
Sortino Ratio                        0.532475
Calmar Ratio                         0.561848
Max. Drawdown [%]                  -15.950773
Avg. Drawdown [%]                   -6.247143
Max. Drawdown Duration       53 days 00:00:00
Avg. Drawdown Duration       19 days 00:00:00
# Trades                                    3
Win Rate [%]                        33.333333
Best Trade [%]                      10.367128
Worst Trade [%]                     -0.979151
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2457,137,160,406.97545,409.0,-4974.31935,-0.004975,2022-04-07,2022-05-16,39 days
1,2420,160,162,411.02455,407.0,-9739.41100,-0.009792,2022-05-16,2022-05-18,2 days
2,-2432,162,245,404.98535,363.0,102108.37120,0.103671,2022-05-18,2022-09-14,119 days


<Strategy EmaCrossStrategy_WithShortPosition(EMAshort=40,EMAlong=60)>

**SMA**

In [20]:
import BacktestingStrategies.Strategy_SMACross as btstSMA
importlib.reload(btstSMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMA.SmaCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    EMAshort=range(10, 100, 5),\
    EMAlong=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000,\
    constraint = lambda x: x.EMAshort < x.EMAlong)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstSMA.SmaCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        EMAshort=range(10, 100, 5),\
        EMAlong=range(10, 200, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000,\
        constraint = lambda x: x.EMAshort < x.EMAlong)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   44.715447
Equity Final [$]                  632539.1893
Equity Peak [$]                  1059085.4268
Return [%]                         -36.746081
Buy & Hold Return [%]                -0.27933
Return (Ann.) [%]                  -37.448762
Volatility (Ann.) [%]               19.530528
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -43.40011
Avg. Drawdown [%]                  -28.896834
Max. Drawdown Duration      189 days 00:00:00
Avg. Drawdown Duration      143 days 00:00:00
# Trades                                    5
Win Rate [%]                              0.0
Best Trade [%]                      -2.110569
Worst Trade [%]                    -11.992242
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2346,53,72,426.09880,380.0,-108147.7848,-0.108188,2021-12-02,2021-12-29,27 days
1,2093,85,128,426.09880,378.0,-100670.7884,-0.112882,2022-01-20,2022-03-25,64 days
2,1830,151,169,432.12850,401.0,-56965.1550,-0.072035,2022-04-27,2022-05-27,30 days
3,1723,188,207,426.09880,375.0,-88043.2324,-0.119922,2022-06-23,2022-07-21,28 days
4,1742,239,245,370.82655,363.0,-13633.8501,-0.021106,2022-09-06,2022-09-14,8 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 513 configurations.
  output = _optimize_grid()


AttributeError: Strategy 'SmaCross' is missing parameter 'EMAshort'.Strategy class should define parameters as class variables before they can be optimized or run with.

**ボリンジャーバンド**

In [15]:
import BacktestingStrategies.Strategy_BBD as btstBBD
importlib.reload(btstBBD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBD.BBsigma, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBD.BBsigma_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   30.894309
Equity Final [$]                1092278.19155
Equity Peak [$]                  1164211.7707
Return [%]                           9.227819
Buy & Hold Return [%]                1.117318
Return (Ann.) [%]                     9.46322
Volatility (Ann.) [%]               22.381908
Sharpe Ratio                         0.422807
Sortino Ratio                        0.741626
Calmar Ratio                         0.666849
Max. Drawdown [%]                  -14.190939
Avg. Drawdown [%]                    -5.93861
Max. Drawdown Duration       89 days 00:00:00
Avg. Drawdown Duration       35 days 00:00:00
# Trades                                    8
Win Rate [%]                             37.5
Best Trade [%]                       14.73262
Worst Trade [%]                     -3.602169
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2512,127,129,397.96020,385.0,-32556.02240,-0.032567,2022-03-24,2022-03-28,4 days
1,2500,129,152,386.90575,423.0,90235.62500,0.093290,2022-03-28,2022-04-28,31 days
2,2740,175,176,385.90080,372.0,-38088.19200,-0.036022,2022-06-06,2022-06-07,1 days
3,2727,176,178,373.84140,366.0,-21383.49780,-0.020975,2022-06-07,2022-06-09,2 days
4,2713,178,185,367.81170,422.0,147012.85790,0.147326,2022-06-09,2022-06-20,11 days
5,3038,207,208,376.85625,370.0,-20829.28750,-0.018193,2022-07-21,2022-07-22,1 days
6,3023,208,209,371.83150,373.0,3532.37550,0.003143,2022-07-22,2022-07-25,3 days
7,3009,209,245,374.84635,363.0,-35645.66715,-0.031603,2022-07-25,2022-09-14,51 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   79.268293
Equity Final [$]                 1341234.0944
Equity Peak [$]                  1401404.0819
Return [%]                          34.123409
Buy & Hold Return [%]                1.117318
Return (Ann.) [%]                   35.087279
Volatility (Ann.) [%]                  62.214
Sharpe Ratio                         0.563977
Sortino Ratio                        1.219095
Calmar Ratio                         1.084343
Max. Drawdown [%]                  -32.358105
Avg. Drawdown [%]                   -6.623797
Max. Drawdown Duration       97 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                    6
Win Rate [%]                        66.666667
Best Trade [%]                      10.695383
Worst Trade [%]                       -3.6768
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2537,51,127,394.03980,396.0,-4973.0274,-0.004975,2021-11-30,2022-03-24,114 days
1,2500,127,152,397.96020,423.0,62599.5000,0.062920,2022-03-24,2022-04-28,35 days
2,-2512,152,175,420.90615,384.0,92708.2488,0.087683,2022-04-28,2022-06-06,39 days
3,2980,175,185,385.90080,422.0,107575.6160,0.093545,2022-06-06,2022-06-20,14 days
4,-2995,185,207,419.91110,375.0,134508.7445,0.106954,2022-06-20,2022-07-21,31 days
5,3694,207,245,376.85625,363.0,-51184.9875,-0.036768,2022-07-21,2022-09-14,55 days


**RSI**

In [9]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstRsi.RsiOscillator_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2016-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                   2191 days 00:00:00
Exposure Time [%]                   61.616162
Equity Final [$]                 317185.69525
Equity Peak [$]                 1023461.34405
Return [%]                          -68.28143
Buy & Hold Return [%]              -87.567763
Return (Ann.) [%]                  -17.704815
Volatility (Ann.) [%]               21.696617
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -75.690733
Avg. Drawdown [%]                  -23.794461
Max. Drawdown Duration     1978 days 00:00:00
Avg. Drawdown Duration      521 days 00:00:00
# Trades                                   32
Win Rate [%]                             25.0
Best Trade [%]                      20.969827
Worst Trade [%]                     -34.32744
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,493,72,75,2026.98415,1940.0,-42883.18595,-0.042913,2016-12-30,2017-01-06,7 days
1,490,75,162,1949.60300,1753.0,-96335.47000,-0.100843,2017-01-06,2017-05-16,130 days
2,488,162,164,1761.67735,1827.0,31877.45320,0.037080,2017-05-16,2017-05-18,2 days
3,486,164,177,1836.04365,1717.0,-57855.21390,-0.064837,2017-05-18,2017-06-06,19 days
4,483,177,253,1725.49915,1636.0,-43228.08945,-0.051869,2017-06-06,2017-09-21,107 days
5,481,253,255,1644.09820,1643.0,-528.23420,-0.000668,2017-09-21,2017-09-25,4 days
6,479,255,257,1651.13285,1648.0,-1500.63515,-0.001897,2017-09-25,2017-09-27,2 days
7,476,257,260,1656.15760,1626.0,-14355.01760,-0.018209,2017-09-27,2017-10-02,5 days
8,474,260,291,1634.04870,1343.0,-137957.08380,-0.178115,2017-10-02,2017-11-14,43 days
9,472,291,335,1349.64785,1165.0,-87153.78520,-0.136812,2017-11-14,2018-01-15,62 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2016-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                   2191 days 00:00:00
Exposure Time [%]                   39.461279
Equity Final [$]                 1281601.2448
Equity Peak [$]                  1281601.2448
Return [%]                          28.160124
Buy & Hold Return [%]              -87.567763
Return (Ann.) [%]                    4.300249
Volatility (Ann.) [%]               24.117225
Sharpe Ratio                         0.178306
Sortino Ratio                        0.308416
Calmar Ratio                         0.095258
Max. Drawdown [%]                  -45.143013
Avg. Drawdown [%]                  -22.012413
Max. Drawdown Duration      447 days 00:00:00
Avg. Drawdown Duration      170 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      28.176502
Worst Trade [%]                     28.176502
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,758,290,875,1318.4944,1690.0,281601.2448,0.281765,2017-11-13,2020-03-17,855 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,392,25,37,2545.53835,2676.45,51317.36680,0.051428,2016-10-21,2016-11-09,19 days
1,-423,38,41,2481.65470,2350.00,55689.93810,0.053051,2016-11-10,2016-11-15,5 days
2,468,41,138,2361.63250,1999.00,-169712.01000,-0.153552,2016-11-15,2017-04-07,143 days
3,-471,138,151,1989.10495,1897.00,43381.43145,0.046305,2017-04-07,2017-04-26,19 days
4,514,151,223,1906.39015,1762.00,-74216.53710,-0.075740,2017-04-26,2017-08-10,106 days
5,-517,223,248,1753.27810,1742.00,5830.77770,0.006433,2017-08-10,2017-09-14,35 days
6,521,248,351,1750.62290,1389.00,-188405.53090,-0.206568,2017-09-14,2018-02-06,145 days
7,-523,351,403,1382.12445,1258.00,64917.08735,0.089807,2018-02-06,2018-04-19,72 days
8,623,403,433,1264.22710,1260.00,-2633.48330,-0.003344,2018-04-19,2018-05-31,42 days
9,-627,433,439,1253.76300,1191.00,39352.40100,0.050060,2018-05-31,2018-06-08,8 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2016-09-13 00:00:00
End                       2022-09-13 00:00:00
Duration                   2191 days 00:00:00
Exposure Time [%]                   99.191919
Equity Final [$]                  4857527.082
Equity Peak [$]                   5002646.082
Return [%]                         385.752708
Buy & Hold Return [%]              -87.567763
Return (Ann.) [%]                    30.76231
Volatility (Ann.) [%]              114.488082
Sharpe Ratio                         0.268694
Sortino Ratio                        0.801302
Calmar Ratio                         0.361459
Max. Drawdown [%]                  -85.105853
Avg. Drawdown [%]                   -6.903352
Max. Drawdown Duration      339 days 00:00:00
Avg. Drawdown Duration       32 days 00:00:00
# Trades                                   23
Win Rate [%]                         78.26087
Best Trade [%]                      28.231703
Worst Trade [%]                    -13.040789
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-364,12,61,2746.33800,1971.00,282223.03200,0.282317,2016-10-03,2016-12-14,72 days
1,647,61,79,1980.75645,1973.00,-5018.42315,-0.003916,2016-12-14,2017-01-13,30 days
2,-650,79,157,1963.23365,1759.00,132751.87250,0.104029,2017-01-13,2017-05-09,116 days
3,797,157,200,1767.70705,1757.00,-8533.51885,-0.006057,2017-05-09,2017-07-07,59 days
4,-801,200,263,1748.30285,1586.00,130004.58285,0.092835,2017-07-07,2017-10-05,90 days
5,960,263,293,1593.85070,1386.00,-199536.67200,-0.130408,2017-10-05,2017-11-16,42 days
6,-965,293,330,1379.13930,1177.00,195064.42450,0.146569,2017-11-16,2018-01-08,53 days
7,1290,330,347,1182.82615,1227.00,56984.26650,0.037346,2018-01-08,2018-01-31,23 days
8,-1297,347,515,1220.92635,1056.00,213909.47595,0.135083,2018-01-31,2018-09-24,236 days
9,1694,515,527,1061.22720,1072.00,18249.12320,0.010151,2018-09-24,2018-10-10,16 days


<Strategy RsiOscillator_WithShortPosition(upper_bound=50,lower_bound=20,rsi_window=10)>